---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [6]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt



---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [7]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [9]:
# # G = nx.barabasi_albert_graph(1000000,1)
# for j in range(5):
#     degrees = P1_Graphs[j].degree()
#     degree_values = sorted(set(degrees.values()))
#     histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(P1_Graphs[j])) for i in degree_values]
#     plt.plot(degree_values,histogram, 'o')
#     plt.xlabel('Degree')
#     plt.ylabel('Fraction of Nodes')
#     plt.xscale('log')
#     plt.yscale('log')
#     plt.show()

In [10]:
Q1_ans = ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

In [11]:
def graph_identification():
    
    # Your Code Here
    
    return Q1_ans# Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [13]:
G = nx.read_gpickle('email_prediction.txt')

# print(nx.info(G))
G

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [46]:
nodes_df = pd.DataFrame(G.nodes())
nodes_df.columns = ['nodes']
# Q4_df.columns = ['staff3','staff2','weight']

# degree = pd.DataFrame(G.degree())
# G.degree()
# test = G.degree()
degree_df = pd.DataFrame.from_dict(G.degree(),orient='index')
# degree_df.head()
# type(test)
degree_df.columns = ['degree']


degCent = nx.degree_centrality(G)
# degree_centrality = degCent[100]
degCent_df = pd.DataFrame.from_dict(degCent,orient='index')
degCent_df.columns = ['degCent']


closeCent = nx.closeness_centrality(G, normalized = False)
# closeness_centrality = closeCent[100]
closeCent_df = pd.DataFrame.from_dict(closeCent,orient='index')
closeCent_df.columns = ['closeCent']


btwnCent = nx.betweenness_centrality(G,normalized = True, endpoints = False)
# betweenness_centrality = btwnCent[100]
btwnCent_df = pd.DataFrame.from_dict(btwnCent,orient='index')
btwnCent_df.columns = ['btwnCent']

# degCent

In [47]:
page_rank = nx.pagerank(G, alpha=0.85)
# page_rank
page_rank_df = pd.DataFrame.from_dict(page_rank,orient='index')
page_rank_df.columns = ['page_rank']


In [48]:
HITS = nx.hits(G)
hub_df = pd.DataFrame.from_dict(HITS[0],orient='index')
authority_df = pd.DataFrame.from_dict(HITS[1],orient='index')
hub_df.columns = ['hub']
authority_df.columns = ['authority']


In [60]:


# authority_df.head()
# degree_df.head()

,nodes,degree,degCent,closeCent,btwnCent,page_rank,hub,authority
1000,1000,6,0.005976,0.362799,0.000004,0.000294,0.000161,0.000161
1001,1001,10,0.009960,0.346343,0.000004,0.000398,0.000194,0.000194
1002,1002,1,0.000996,0.303731,0.000000,0.000176,0.000017,0.000017
1003,1003,1,0.000996,0.303919,0.000000,0.000177,0.000024,0.000024
1004,1004,1,0.000996,0.301685,0.000000,0.000175,0.000017,0.000017


In [66]:
nodes = G.nodes()
Q2_test = nx.get_node_attributes(G, 'ManagementSalary')
Q2_test
ManagementSalary_df = pd.DataFrame.from_dict(Q2_test,orient='index')
ManagementSalary_df.columns = ['ManagementSalary']

# # attributes
# nodes
# attributes = []
# for i in nodes:
#     attributes.append([i,nx.get_node_attributes(G,i)])
# attributes
# # attributes_df = pd.DataFrame(attributes)
# # attributes_df.head()


In [67]:
Q2_df = pd.concat([nodes_df, degree_df, degCent_df, closeCent_df, btwnCent_df, page_rank_df, hub_df, authority_df, ManagementSalary_df], axis=1)
Q2_df.tail()

,nodes,degree,degCent,closeCent,btwnCent,page_rank,hub,authority,ManagementSalary
1000,1000,6,0.005976,0.362799,0.000004,0.000294,0.000161,0.000161,NaN
1001,1001,10,0.009960,0.346343,0.000004,0.000398,0.000194,0.000194,NaN
1002,1002,1,0.000996,0.303731,0.000000,0.000176,0.000017,0.000017,0.0
1003,1003,1,0.000996,0.303919,0.000000,0.000177,0.000024,0.000024,0.0
1004,1004,1,0.000996,0.301685,0.000000,0.000175,0.000017,0.000017,0.0


In [68]:
Q2_df.to_csv('4th_assignment_Q2', index=False)
Q2_df.to_csv('4th_assignment_Q2_true', index=True)

In [75]:
# Q2_trim_df = Q2_df.drop(['nodes', 'degree'], axis = 1)
# Q2_trim_df.tail()

In [89]:
Q2_test = Q2_df[Q2_df['ManagementSalary'].isnull()]
Q2_test.head()

Q2_X_test = Q2_test.drop(['ManagementSalary'],axis = 1)
Q2_X_test = Q2_X_test.set_index('nodes')

In [94]:
Q2_train = Q2_df[Q2_df['ManagementSalary'].notnull()]
Q2_train.head()
Q2_train = Q2_train.set_index('nodes')

Q2_X_train = Q2_train.drop(['ManagementSalary'],axis = 1)
Q2_y_train = Q2_train['ManagementSalary']
# Q2_y_train
Q2_X_train.tail()
# Q2_X_train = Q2_X_train.set_index('nodes')
# Q2_y_train = Q2_y_train.set_index('nodes')

,degree,degCent,closeCent,btwnCent,page_rank,hub,authority
nodes,,,,,,,
998,1,0.000996,0.319805,0.0,0.000176,0.000026,0.000026
999,1,0.000996,0.317742,0.0,0.000172,0.000040,0.000040
1002,1,0.000996,0.303731,0.0,0.000176,0.000017,0.000017
1003,1,0.000996,0.303919,0.0,0.000177,0.000024,0.000024
1004,1,0.000996,0.301685,0.0,0.000175,0.000017,0.000017


In [102]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(Q2_X_train, Q2_y_train)

y_pred=clf.predict_proba(Q2_X_test)
idx = Q2_X_test.index

output_idx = idx.tolist()

output = []
for comp in y_pred:
    output.append(comp[1])
output

type(output_idx)

# data = np.array(['g', 'e', 'e', 'k', 's']) 
  
# # providing an index 
# ser = pd.Series(data, index =[10, 11, 12, 13, 14])

Q2_ans_data = np.array(output)
Q2_ans = pd.Series(Q2_ans_data, index = output_idx)
Q2_ans

1       0.16
2       0.95
5       0.99
8       0.07
14      0.14
18      0.26
27      0.22
30      0.27
31      0.49
34      0.10
37      0.06
40      0.18
45      0.12
54      0.40
55      0.18
60      0.27
62      1.00
65      0.95
77      0.02
79      0.37
97      0.00
101     0.00
103     0.16
108     0.07
113     0.10
122     0.06
141     0.29
142     1.00
144     0.06
145     0.12
        ... 
913     0.00
914     0.00
915     0.00
918     0.02
923     0.01
926     0.00
931     0.01
934     0.00
939     0.00
944     0.00
945     0.00
947     0.25
950     0.36
951     0.00
953     0.00
959     0.00
962     0.00
963     0.24
968     0.00
969     0.00
974     0.00
984     0.00
987     0.00
989     0.00
991     0.00
992     0.00
994     0.00
996     0.00
1000    0.03
1001    0.00
Length: 252, dtype: float64

In [103]:
def salary_predictions():
    
    # Your Code Here
    
    return Q2_ans # Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [139]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [112]:
edges_df = pd.DataFrame(G.edges(), index=G.edges())

edges_df.columns = ['first', 'second']
edges_df.head()

first = edges_df['first'].tolist()
first
second = edges_df['second'].tolist()

common_neigh = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in nx.non_edges(G)]
common_neigh


[(0, 2, 6),
 (0, 3, 3),
 (0, 4, 3),
 (0, 7, 4),
 (0, 8, 1),
 (0, 9, 2),
 (0, 10, 1),
 (0, 11, 4),
 (0, 12, 4),
 (0, 13, 7),
 (0, 14, 11),
 (0, 15, 0),
 (0, 16, 4),
 (0, 19, 4),
 (0, 20, 7),
 (0, 21, 14),
 (0, 22, 0),
 (0, 23, 3),
 (0, 24, 2),
 (0, 25, 1),
 (0, 26, 0),
 (0, 27, 1),
 (0, 28, 2),
 (0, 29, 3),
 (0, 30, 0),
 (0, 31, 0),
 (0, 32, 0),
 (0, 33, 0),
 (0, 34, 1),
 (0, 35, 1),
 (0, 36, 1),
 (0, 37, 0),
 (0, 38, 0),
 (0, 39, 1),
 (0, 40, 1),
 (0, 41, 5),
 (0, 42, 11),
 (0, 43, 1),
 (0, 44, 4),
 (0, 45, 0),
 (0, 46, 1),
 (0, 47, 1),
 (0, 48, 2),
 (0, 49, 1),
 (0, 50, 0),
 (0, 51, 5),
 (0, 52, 6),
 (0, 53, 8),
 (0, 54, 3),
 (0, 55, 3),
 (0, 56, 1),
 (0, 57, 4),
 (0, 58, 7),
 (0, 59, 2),
 (0, 60, 8),
 (0, 61, 12),
 (0, 62, 10),
 (0, 63, 4),
 (0, 66, 4),
 (0, 67, 2),
 (0, 68, 1),
 (0, 69, 2),
 (0, 70, 0),
 (0, 71, 0),
 (0, 72, 0),
 (0, 75, 1),
 (0, 76, 1),
 (0, 77, 2),
 (0, 78, 1),
 (0, 79, 5),
 (0, 80, 3),
 (0, 81, 5),
 (0, 82, 14),
 (0, 83, 7),
 (0, 84, 7),
 (0, 85, 11),
 (0, 86, 19

In [130]:
common_neigh_df = pd.DataFrame(common_neigh)
common_neigh_df.head()
common_neigh_df.columns = ['first', 'second', 'common_neigh']

# Q3_common_neigh_idx = tuple(common_neigh_df['first'].tolist(), common_neigh_df['second'].tolist())

def listOfTuples(l1, l2): 
    return list(map(lambda x, y:(x,y), l1, l2)) 
  
# Driver Code 
# list1 = [1, 2, 3] 
# list2 = ['a', 'b', 'c'] 
  
# print(listOfTuples(list1, list2))



common_neigh_df['idx'] = listOfTuples(common_neigh_df['first'].tolist(), common_neigh_df['second'].tolist())

common_neigh_df = common_neigh_df.set_index('idx')
common_neigh_df.index.name = None

common_neigh_df = common_neigh_df.drop(['first', 'second'],axis = 1)

common_neigh_df.head()



,common_neigh
"(0, 2)",6
"(0, 3)",3
"(0, 4)",3
"(0, 7)",4
"(0, 8)",1


In [131]:
Jaccard_Coefficient = list(nx.jaccard_coefficient(G))

Jaccard_Coefficient_df = pd.DataFrame(Jaccard_Coefficient)
Jaccard_Coefficient_df.head()
Jaccard_Coefficient_df.columns = ['first', 'second', 'Jaccard_Coefficient']

# Q3_common_neigh_idx = tuple(common_neigh_df['first'].tolist(), common_neigh_df['second'].tolist())

def listOfTuples(l1, l2): 
    return list(map(lambda x, y:(x,y), l1, l2)) 
  
# Driver Code 
# list1 = [1, 2, 3] 
# list2 = ['a', 'b', 'c'] 
  
# print(listOfTuples(list1, list2))



Jaccard_Coefficient_df['idx'] = listOfTuples(Jaccard_Coefficient_df['first'].tolist(), Jaccard_Coefficient_df['second'].tolist())

Jaccard_Coefficient_df = Jaccard_Coefficient_df.set_index('idx')
Jaccard_Coefficient_df.index.name = None

Jaccard_Coefficient_df = Jaccard_Coefficient_df.drop(['first', 'second'],axis = 1)

Jaccard_Coefficient_df.head()



,Jaccard_Coefficient
"(0, 2)",0.045802
"(0, 3)",0.027273
"(0, 4)",0.022222
"(0, 7)",0.036364
"(0, 8)",0.012821


In [132]:
resource_allocation = list(nx.resource_allocation_index(G))

resource_allocation_df = pd.DataFrame(resource_allocation)
resource_allocation_df.head()
resource_allocation_df.columns = ['first', 'second', 'resource_allocation']

# Q3_common_neigh_idx = tuple(common_neigh_df['first'].tolist(), common_neigh_df['second'].tolist())

def listOfTuples(l1, l2): 
    return list(map(lambda x, y:(x,y), l1, l2)) 
  
# Driver Code 
# list1 = [1, 2, 3] 
# list2 = ['a', 'b', 'c'] 
  
# print(listOfTuples(list1, list2))



resource_allocation_df['idx'] = listOfTuples(resource_allocation_df['first'].tolist(), resource_allocation_df['second'].tolist())

resource_allocation_df = resource_allocation_df.set_index('idx')
resource_allocation_df.index.name = None

resource_allocation_df = resource_allocation_df.drop(['first', 'second'],axis = 1)


resource_allocation_df.head()



,resource_allocation
"(0, 2)",0.055340
"(0, 3)",0.021388
"(0, 4)",0.021388
"(0, 7)",0.061668
"(0, 8)",0.011628


In [133]:
adamic_adar = list(nx.adamic_adar_index(G))

adamic_adar_df = pd.DataFrame(adamic_adar)
adamic_adar_df.head()
adamic_adar_df.columns = ['first', 'second', 'adamic_adar']

# Q3_common_neigh_idx = tuple(common_neigh_df['first'].tolist(), common_neigh_df['second'].tolist())

def listOfTuples(l1, l2): 
    return list(map(lambda x, y:(x,y), l1, l2)) 
  
# Driver Code 
# list1 = [1, 2, 3] 
# list2 = ['a', 'b', 'c'] 
  
# print(listOfTuples(list1, list2))



adamic_adar_df['idx'] = listOfTuples(adamic_adar_df['first'].tolist(), adamic_adar_df['second'].tolist())

adamic_adar_df = adamic_adar_df.set_index('idx')
adamic_adar_df.index.name = None

adamic_adar_df = adamic_adar_df.drop(['first', 'second'],axis = 1)

adamic_adar_df.head()



,adamic_adar
"(0, 2)",1.257626
"(0, 3)",0.605728
"(0, 4)",0.605728
"(0, 7)",0.938816
"(0, 8)",0.224500


In [134]:
preferential_attachment = list(nx.preferential_attachment(G))

preferential_attachment_df = pd.DataFrame(preferential_attachment)
preferential_attachment_df.head()
preferential_attachment_df.columns = ['first', 'second', 'preferential_attachment']

# Q3_common_neigh_idx = tuple(common_neigh_df['first'].tolist(), common_neigh_df['second'].tolist())

def listOfTuples(l1, l2): 
    return list(map(lambda x, y:(x,y), l1, l2)) 
  
# Driver Code 
# list1 = [1, 2, 3] 
# list2 = ['a', 'b', 'c'] 
  
# print(listOfTuples(list1, list2))



preferential_attachment_df['idx'] = listOfTuples(preferential_attachment_df['first'].tolist(), preferential_attachment_df['second'].tolist())

preferential_attachment_df = preferential_attachment_df.set_index('idx')
preferential_attachment_df.index.name = None


preferential_attachment_df = preferential_attachment_df.drop(['first', 'second'],axis = 1)

preferential_attachment_df.head()



,preferential_attachment
"(0, 2)",4180
"(0, 3)",3124
"(0, 4)",4224
"(0, 7)",3168
"(0, 8)",1628


In [126]:
# cn = list(nx.cn_soundarajan_hopcroft(G)) 
# # cn: community common neighbors

# cn_df = pd.DataFrame(cn)
# cn_df.head()
# cn_df.columns = ['first', 'second', 'cn']

# # Q3_common_neigh_idx = tuple(common_neigh_df['first'].tolist(), common_neigh_df['second'].tolist())

# def listOfTuples(l1, l2): 
#     return list(map(lambda x, y:(x,y), l1, l2)) 
  
# # Driver Code 
# # list1 = [1, 2, 3] 
# # list2 = ['a', 'b', 'c'] 
  
# # print(listOfTuples(list1, list2))



# cn_df['idx'] = listOfTuples(cn_df['first'].tolist(), cn_df['second'].tolist())

# cn_df = cn_df.set_index('idx')
# cn_df.index.name = None
# cn_df.head()

In [ ]:
# common_neigh = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0],
# e[1])))) for e in nx.non_edges(G)]

In [128]:
# cr = list(nx.ra_index_soundarajan_hopcroft(G)) # cr: Community Resource Allocation

# cr_df = pd.DataFrame(cr)
# cr_df.head()
# cr_df.columns = ['first', 'second', 'cr']

# # Q3_common_neigh_idx = tuple(common_neigh_df['first'].tolist(), common_neigh_df['second'].tolist())

# def listOfTuples(l1, l2): 
#     return list(map(lambda x, y:(x,y), l1, l2)) 
  
# # Driver Code 
# # list1 = [1, 2, 3] 
# # list2 = ['a', 'b', 'c'] 
  
# # print(listOfTuples(list1, list2))



# cr_df['idx'] = listOfTuples(cr_df['first'].tolist(), cr_df['second'].tolist())

# cr_df = cr_df.set_index('idx')
# cr_df.index.name = None
# cr_df.head()

In [137]:
Q3_df = pd.concat([common_neigh_df, Jaccard_Coefficient_df, resource_allocation_df, adamic_adar_df, preferential_attachment_df], axis=1)
Q3_df.head()



,common_neigh,Jaccard_Coefficient,resource_allocation,adamic_adar,preferential_attachment
"(0, 2)",6,0.045802,0.055340,1.257626,4180
"(0, 3)",3,0.027273,0.021388,0.605728,3124
"(0, 4)",3,0.022222,0.021388,0.605728,4224
"(0, 7)",4,0.036364,0.061668,0.938816,3168
"(0, 8)",1,0.012821,0.011628,0.224500,1628


In [141]:
Q3_df.to_csv('4th_assignment_Q3', index=False)
Q3_df.to_csv('4th_assignment_Q3_true', index=True)

In [140]:
Q3_data_df = pd.merge(future_connections, Q3_df, how = 'inner', left_index=True, right_index=True)
Q3_data_df.head()

,Future Connection,common_neigh,Jaccard_Coefficient,resource_allocation,adamic_adar,preferential_attachment
"(6, 840)",0.0,9,0.073770,0.136721,2.110314,2070
"(4, 197)",0.0,2,0.015504,0.008437,0.363528,3552
"(620, 979)",0.0,0,0.000000,0.000000,0.000000,28
"(519, 872)",0.0,2,0.060606,0.039726,0.507553,299
"(382, 423)",0.0,0,0.000000,0.000000,0.000000,205


In [146]:
Q3_test = Q3_data_df[Q3_data_df['Future Connection'].isnull()]
Q3_test.head()
Q3_train = Q3_data_df[Q3_data_df['Future Connection'].notnull()]
Q3_train.head()

# Q2_X_test = Q2_test.drop(['ManagementSalary'],axis = 1)
# Q2_X_test = Q2_X_test.set_index('nodes')

,Future Connection,common_neigh,Jaccard_Coefficient,resource_allocation,adamic_adar,preferential_attachment
"(6, 840)",0.0,9,0.073770,0.136721,2.110314,2070
"(4, 197)",0.0,2,0.015504,0.008437,0.363528,3552
"(620, 979)",0.0,0,0.000000,0.000000,0.000000,28
"(519, 872)",0.0,2,0.060606,0.039726,0.507553,299
"(382, 423)",0.0,0,0.000000,0.000000,0.000000,205


In [149]:
Q3_X_test = Q3_test.drop(['Future Connection'],axis = 1)
# Q2_X_test = Q2_X_test.set_index('nodes')


# Q2_train = Q2_df[Q2_df['ManagementSalary'].notnull()]
# Q2_train.head()
# Q2_train = Q2_train.set_index('nodes')

Q3_X_train = Q3_train.drop(['Future Connection'],axis = 1)
Q3_y_train = Q3_train['Future Connection']
# Q3_y_train
Q3_X_train.head()
# Q2_X_train = Q2_X_train.set_index('nodes')
# Q2_y_train = Q2_y_train.set_index('nodes')

,common_neigh,Jaccard_Coefficient,resource_allocation,adamic_adar,preferential_attachment
"(6, 840)",9,0.073770,0.136721,2.110314,2070
"(4, 197)",2,0.015504,0.008437,0.363528,3552
"(620, 979)",0,0.000000,0.000000,0.000000,28
"(519, 872)",2,0.060606,0.039726,0.507553,299
"(382, 423)",0,0.000000,0.000000,0.000000,205


In [151]:
# # MinMaxScaler
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(0, 1))
# Q3_X_train_rescaled = scaler.fit_transform(Q3_X_train)
# Q3_X_test_rescaled = scaler.fit_transform(Q3_X_test)

In [153]:
from sklearn.preprocessing import Normalizer
# scaler = Normalizer().fit(X)
# normalizedX = scaler.transform(X)
scaler = Normalizer().fit(Q3_X_train)
Q3_X_train_rescaled = scaler.transform(Q3_X_train)
Q3_X_test_rescaled = scaler.transform(Q3_X_test)

In [154]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB



Q3_clf = RandomForestClassifier(n_estimators=100)

# Q3_clf = GaussianNB()

# Q3_clf.fit(Q3_X_train, Q3_y_train)
# Q3_y_pred = Q3_clf.predict_proba(Q3_X_test)


Q3_clf.fit(Q3_X_train_rescaled, Q3_y_train)
Q3_y_pred = Q3_clf.predict_proba(Q3_X_test_rescaled)



Q3_idx = Q3_X_test.index

Q3_output_idx = Q3_idx.tolist()

Q3_output = []
for comp in Q3_y_pred:
    Q3_output.append(comp[1])
# output

# type(output_idx)

# data = np.array(['g', 'e', 'e', 'k', 's']) 
  
# # providing an index 
# ser = pd.Series(data, index =[10, 11, 12, 13, 14])

Q3_ans_data = np.array(Q3_output)
Q3_ans = pd.Series(Q3_ans_data, index = Q3_output_idx)
Q3_ans

(107, 348)    0.292036
(542, 751)    0.012729
(20, 426)     0.297801
(50, 989)     0.012729
(942, 986)    0.012729
(324, 857)    0.012729
(13, 710)     0.279197
(19, 271)     0.112661
(319, 878)    0.012729
(659, 707)    0.012729
(49, 843)     0.012729
(208, 893)    0.012729
(377, 469)    0.012729
(405, 999)    0.000000
(129, 740)    0.114666
(292, 618)    0.060000
(239, 689)    0.012729
(359, 373)    0.012729
(53, 523)     0.010805
(276, 984)    0.012729
(202, 997)    0.012729
(604, 619)    0.075000
(270, 911)    0.012729
(261, 481)    0.056769
(200, 450)    0.485313
(213, 634)    0.012729
(644, 735)    0.240000
(346, 553)    0.012729
(521, 738)    0.012729
(422, 953)    0.036797
                ...   
(672, 848)    0.012729
(28, 127)     0.877677
(202, 661)    0.012729
(54, 195)     0.970000
(295, 864)    0.012729
(814, 936)    0.012729
(839, 874)    0.012729
(139, 843)    0.012729
(461, 544)    0.012729
(68, 487)     0.012729
(622, 932)    0.012729
(504, 936)    0.012117
(479, 528) 

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    
    return Q3_ans# Your Answer Here